In [ ]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from joblib import Parallel, delayed
import matplotlib.pyplot as plt
from sklearn.utils import resample
from scipy.stats import norm
from sklearn.model_selection import train_test_split
import multiprocessing
import matplotlib.pyplot as plt
import time

In [ ]:
# calculate r-separation distance of dataset
def get_nearest_oppo_dist(X, y, norm, n_jobs):
    if len(X.shape) > 2:
        X = X.reshape(len(X), -1)
    p = norm

    def helper(yi):
        return NearestNeighbors(n_neighbors=1, 
                                metric='minkowski', p=p, n_jobs=-1).fit(X[y != yi])

    nns = Parallel(n_jobs=n_jobs)(delayed(helper)(yi) for yi in np.unique(y))
    ret = np.zeros(len(X))
    for yi in np.unique(y):
        dist, _ = nns[yi].kneighbors(X[y == yi], n_neighbors=1)
        ret[np.where(y == yi)[0]] = dist[:, 0]

    return nns, ret

In [ ]:
#sampling function for corruptions within a L_inf distance around every datapoint
def sampling(position, label, distance, k_samples, seed):
    np.random.seed(seed)
    x_low = position - distance
    x_high = position + distance
    x_samples = np.random.uniform(x_low, x_high, (k_samples,2))
    sample_list = []
    
    for x_sample in x_samples:
        sample_list.append([x_sample[0], x_sample[1], label])

    return sample_list

In [ ]:
#sampling function for corruptions within a L_2 distance around every datapoint
# small k sampling can go wrong when in some seed all samples are excluded. e.g. k=1 will go wrong
def sampling_round(position, label, distance, k_samples, seed):
    #position = np.array([1, 2])
    #distance = 0.1
    np.random.seed(seed)
    x_low = position - distance
    x_high = position + distance
    x_samples = np.random.uniform(x_low, x_high, [k_samples,2])
    df_samples = pd.DataFrame(x_samples) 
    round_samples = []
    for x1,x2 in zip(df_samples[0],df_samples[1]):
        d = math.sqrt((x1-position[0])*(x1-position[0])+(x2-position[1])*(x2-position[1]))
        if d < distance:
            round_samples.append([x1,x2,label])
        
    return round_samples
    
#s = sampling_round(np.array([1, 1]), 0, 0.1, 1000)
#sdf = pd.DataFrame(s)
#print(sdf)
#plt.scatter(sdf.iloc[:,0], sdf.iloc[:,1], color='black')

In [ ]:
with np.load('./data_2D/B_sep.npz') as dataset:
    x = dataset['x']
    y = dataset['y']
print("Number of Datapoints in the set:%f" % len(x))
time0 = time.perf_counter()
dist = np.inf #1, 2, np.inf

#calculate the minimum distance of any points from different classes
nns, ret = get_nearest_oppo_dist(x, y, dist, -1)
#return minimum and mean value
print("2R-Separation Minimal: %f" % ret.min())
print("2R-Separation Mean: %f" % ret.mean())
#setting corner case corruption distance to half the separation distance
epsilon = ret.min()/2
print("Epsilon: %f" % epsilon)
time1 = time.perf_counter()
print(f"Separation Calculation took {time1 - time0:0.3f} seconds")

In [ ]:
#plot the 2D dataset
def plot_label_clusters(latent_data, labels):
    plt.figure(figsize=(12, 10))
    labels = pd.DataFrame(labels)
    colors = {0:'red', 1:'green'}
    plt.scatter(latent_data[:, 0], latent_data[:, 1], c=labels[0].map(colors))
    #cbar = plt.colorbar()
    #cbar.ax.tick_params(labelsize=15)
    plt.xlabel("x[0]",fontsize=20)
    plt.ylabel("x[1]",fontsize=20)
    plt.title('2D Dataset B',fontsize=25)
    plt.xticks(fontsize=15)
    plt.yticks(fontsize=15)
    plt.savefig("B_sep_data_scatter.svg",bbox_inches='tight')
    plt.show()


plot_label_clusters(x, y)

In [ ]:
timestart = time.perf_counter()
#training corruptions
model_epsilons = [0, 0.001, 0.002, epsilon, 0.007, 0.01, 0.015, 0.02, 0.03
                 ]
#test corruptions. for MSCR calculation, 0 and epsilon are always needed!
eval_epsilons = [0, 0.001, 0.002, epsilon, 0.007, 0.01, 0.015, 0.02, 0.03
                ]
model_epsilons_str = ', '.join(map(str, model_epsilons))
eval_epsilons_str = ', '.join(map(str, eval_epsilons))
avg_test_acc = np.empty([len(eval_epsilons), len(model_epsilons)])
std_test_acc = np.empty([len(eval_epsilons), len(model_epsilons)])

#mscr = "minimal separation corruption robustness" = relative change in accuracy when adding random uniform noise
#of max distance epsilon to the test data in a way that classes stay separated --> avoidable loss
avg_mscr = np.empty([len(model_epsilons)])
std_mscr = np.empty([len(model_epsilons)])

for idm, model_epsilon in enumerate(model_epsilons):
    print("Model Epsilon =", model_epsilon)
    
    runs = 1200
    clearresult = np.empty([runs])
    epsilonresult = np.empty([runs])
    results_acc = np.empty([len(eval_epsilons),runs])

    for seed in range(runs):
        onerun2 = time.perf_counter()
        if idm == 0 and seed == 1:
                print(f"One run on the first model_epsilon took {(onerun2 - onerun):.2f} seconds, all runs on all model_epsilons take about {(onerun2-onerun)*runs*len(model_epsilons):.0f} seconds plus some sampling time in case first model has model_epsilon=0.")
        onerun = time.perf_counter()
        
        with np.load('./data_2D/B_sep.npz') as dataset:
            x = dataset['x']
            y = dataset['y']
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=seed)

        #different models available
        clf = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=seed) 
        #clf = KNeighborsClassifier(n_neighbors = 1)
        #clf = SVC(gamma='auto')

        #time2 = time.perf_counter()
        if model_epsilon == 0:
            clf.fit(x_train, y_train)  
        else:
            k_samples_train = 10
            aug_list_train = []
            for p,y in zip(x_train,y_train):   
                aug_list_train.extend(sampling(p, y, model_epsilon, k_samples_train, seed))    
            df_aug_train = pd.DataFrame(aug_list_train)
            x_aug_train = df_aug_train.iloc[:,0:2]
            y_aug_train = df_aug_train.iloc[:,2]
            clf.fit(x_aug_train, y_aug_train)

        for ide, eval_epsilon in enumerate(eval_epsilons):

            if eval_epsilon == 0:
                acc = clf.score(x_test, y_test)
                results_acc[ide, seed] = acc
            else:
                aug_list_test = []
                k_samples_test = 10
                for p,y in zip(x_test,y_test):   
                    aug_list_test.extend(sampling(p, y, eval_epsilon, k_samples_test, seed))    
                df_aug_test = pd.DataFrame(aug_list_test)
                x_aug_test = df_aug_test.iloc[:,0:2]
                y_aug_test = df_aug_test.iloc[:,2]
                acc = clf.score(x_aug_test, y_aug_test)
                results_acc[ide, seed] = acc
                
    #extra for-loop to get and write down the mean over all runs for every evaluation epsilon of one model_epsilon before training the next model_epsilon:
    for ide2, eval_epsilon2 in enumerate(eval_epsilons):
        avg_test_acc[ide2, idm] = results_acc[ide2,:].mean()*100    
        std_test_acc[ide2, idm] = results_acc[ide2,:].std()*100 
        if eval_epsilon2 == 0:
            clearresult = results_acc[ide2,:]
        if eval_epsilon2 == epsilon:
            epsilonresult = results_acc[ide2,:]
            
    #mscr = "minimal separation unrobustness" = increase in error rate when adding random noise to the test data 
    # in a way that classes stay separated
    avg_mscr[idm] = (clearresult.mean() - epsilonresult.mean())*100
    std_mscr[idm] = np.subtract(clearresult, epsilonresult).std()*100
    
    timeend = time.perf_counter()
    print(f"Training round {idm+1} of {len(model_epsilons)} done after {timeend - timestart:0.2f} seconds")
          
print(avg_test_acc)
print(std_test_acc)
print(avg_mscr)
print(std_mscr)
np.savetxt(
    './results/avg_testacc_augmented.csv',
    avg_test_acc, fmt='%1.4f', delimiter=';', header='Networks trained with'
    ' corruptions (epsilon = {}) along columns THEN evaluated on training set using A-TRSM (epsilon = {}) '
    ' along rows'.format(model_epsilons_str, eval_epsilons_str))

np.savetxt(
    './results/avg_mscr.csv',
    avg_mscr, fmt='%1.4f', delimiter=';', header='Networks trained with'
    ' corruptions (epsilon = {}) along columns. mscr is the difference in test accuracy in percentage points between' 
    'the model evaluated on the normal test dataset and the model evaluated on an augmented testdataset' 
    'where augmentation distance equaling half the minimal class separation distance'.format(model_epsilons_str))

np.savetxt(
    './results/std_testacc_augmented.csv',
    std_test_acc, fmt='%1.4f', delimiter=';', header='Networks trained with'
    ' corruptions (epsilon = {}) along columns THEN evaluated on training set using A-TRSM (epsilon = {}) '
    ' along rows'.format(model_epsilons_str, eval_epsilons_str))

np.savetxt(
    './results/std_mscr.csv',
    std_mscr, fmt='%1.4f', delimiter=';', header='Networks trained with'
    ' corruptions (epsilon = {}) along columns. mscr is the difference in test accuracy in percentage points between' 
    'the model evaluated on the normal test dataset and the model evaluated on an augmented testdataset' 
    'where augmentation distance equaling half the minimal class separation distance'.format(model_epsilons_str))


In [ ]:
#this calculates the convergence of average mscr over runs to illustrate the runs needed to equal out randomization
mscr = np.zeros(runs)
mscr_series = np.zeros(runs)

for i in range(runs):
    mscr[i] = clearresult[i]*100 - epsilonresult[i]*100
    mscr_series[i] = mscr[:i].mean()
    
import matplotlib.pyplot as plt 

x1 = list(range(1,runs+1))
y1 = mscr_series


plt.scatter(x1, y1)
plt.xlabel("Runs")
plt.ylabel("Average mscr")
plt.title("Convergence of average mscr (Model trained on last model_epsilon)")
plt.show()